In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import networkx as nx
import sys
sys.path.append('/home/carlo/workspace/networkqit')
import matplotlib.pyplot as plt
import networkqit as nq
import random
import matplotlib
from wilson2 import Wilson, draw_sampling, trace_estimator

In [ ]:
import matplotlib2tikz

In [ ]:
G = nx.from_numpy_array(nx.to_numpy_array(nx.grid_2d_graph(15,15)))
fig, ax = plt.subplots(figsize=(20,20))
q = 1E-1
draw_sampling(G, *Wilson(G,q=q).sample(),ax=ax,cmap = matplotlib.cm.get_cmap('Paired'))
fig.savefig('grid_15_15_wilson_q_01.png')
#matplotlib2tikz.save('grid_10_6_wilson_q_%.3f.tex' % q)

In [ ]:
q = 0.1    
W = Wilson(G, q=q)
F,roots = W.sample()
trace_estimator(G)
plt.savefig('trace_estimator.png')

In [ ]:
G = nx.ring_of_cliques(5, 10)
L = nx.laplacian_matrix(G).toarray()
lambdai = np.linalg.eigvalsh(L)
reps = 5

beta_range = np.logspace(-3, 3, 75)

num_trees = [np.sum(q/(q+lambdai)) for q in beta_range]
num_trees_wilson = [np.mean([len(Wilson(G,q=1/q).sample()[1]) for _ in range(reps)]) for q in beta_range]
entropy_tree_size = [np.nanmean([entropy(np.histogram(Wilson(G,q=q).sample()[1],bins=len(G))[0]) for _ in range(reps)]) for q in beta_range]

# entropy
S = nq.entropy(L=L, beta_range=beta_range)
# log partition function
Tr = np.log([np.sum(np.exp(-beta*np.linalg.eigvalsh(L))) for beta in beta_range])

In [ ]:
plt.semilogx(beta_range,entropy_tree_size,label='S(tree)')
plt.semilogx(beta_range,S,label='$S(\\beta)$')

In [ ]:
# plotting
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,7))
# ax[0].set_title('Entropy vs trace')
# ax[0].semilogx(beta_range,S,label='$S(\\beta)$')
# ax[0].semilogx(beta_range,Tr,label='$Z=Tr[exp(-\\beta L)]$')
# ax[0].legend()
# ax[0].grid()

ax[0].semilogx(1/beta_range, num_trees, label='$s(q)=\sum_i^n \\frac{q}{q+\lambda_i}$')
ax[0].semilogx(1/beta_range, num_trees_wilson, label='$E[R]$')
ax[0].legend()
ax[0].grid()
ax[0].set_title('Trace estimators (n_samples=%d)' % reps)

nx.draw(G,pos=nx.kamada_kawai_layout(G),node_size=10,node_color='black',edge_color='lightgray',ax=ax[1])
ax[1].set_title('Graph')

plt.savefig('trace_estimators.png',bbox_inches='tight')
plt.show()

In [ ]:
G=nx.random_regular_graph(40,50)
#plt.semilogx(beta_range, [entropy(np.exp(-b*nx.laplacian_spectrum(G))) for b in beta_range])
plt.plot(nx.laplacian_spectrum(G))

In [ ]:
nx.laplacian_spectrum(G)

In [ ]:
G = nx.Graph()
G.add_edge(0,1)
plt.semilogx(beta_range,[np.sum(np.exp(-b*nx.laplacian_spectrum(G) )) for b in beta_range])

In [ ]:
n=5
k=17
tol=1E-5
G = nx.ring_of_cliques(n,k)
L = nx.laplacian_matrix(G).toarray()
lambdai = np.linalg.eigvalsh(L)
plt.plot(lambdai,'o')
plt.grid()
multiplicity_k = np.abs(lambdai-np.array([k]*len(L))) <= tol
print(np.sum(multiplicity_k))
print(G.number_of_edges())
(k*(k-1)/2)*n+n

In [ ]:
def ring_of_regular_graphs(n,l,d):
    #n = 20
    #d = n-1
    #l = 2
    G = nx.disjoint_union_all( [nx.random_regular_graph(d,n) for _ in range(l)])
    for i in range(l):
        G.add_edge(n-1,n)
    return G

S5 = nq.entropy( L=nx.laplacian_matrix(ring_of_regular_graphs(20,2,10)).toarray(), beta_range=beta_range)
plt.semilogx(beta_range,S5,label='regular')
S19 = nq.entropy( L=nx.laplacian_matrix(ring_of_regular_graphs(20,2,19)).toarray(), beta_range=beta_range)
plt.semilogx(beta_range,S19,label='clique')

In [ ]:
plt.plot(np.linalg.eigvalsh(nx.laplacian_matrix(ring_of_regular_graphs(20,2,12)).toarray()),'o')

In [ ]:
plt.plot(np.array(range(3,20)),mult)
plt.grid()

In [ ]:
S = nq.entropy(L=L, beta_range=beta_range)
plt.semilogx(beta_range,S,label='S')

In [ ]:
plt.plot([ lambdai[k+1]/lambdai[k] for k in np.arange(1,len(lambdai)-1)])

In [ ]:
Tr = np.array([ np.sum(np.exp(-beta*np.linalg.eigvalsh(L))) for beta in beta_range ])
TrB0 = np.array([ np.sum(1/(beta+beta*np.linalg.eigvalsh(L))) for beta in beta_range ])

plt.semilogx(beta_range,Tr-TrB0)

In [ ]:
from scipy.linalg import expm, logm
plt.semilogx(beta_range, [(np.trace(expm(-beta*L))) for beta in beta_range])

In [ ]:
import sympy as sp
l = [ sp.Symbol('lambda'+str(i),positive=True,real=True) for i in range(4)]
beta = sp.Symbol('beta',positive=True)
S = np.sum([sp.exp(-beta*l[i])*sp.log(sp.exp(-beta*l[i])) for i in range(4)])
sp.diff(S,beta)

In [ ]:
import scipy.special
def nchoosek(n, k):
    return scipy.special.binom(n, k)

In [ ]:
import networkx as nx
import numpy as np
G = nx.karate_club_graph()
A = nx.to_numpy_array(G)
L = nx.laplacian_matrix(G).toarray()
D = np.diag(A.sum(0))
k = 2
print(np.trace(np.linalg.matrix_power(L,k)))
[ np.trace(np.linalg.matrix_power(D,i+1))*np.trace(np.linalg.matrix_power(A,k-i+1)) for i in np.arange(0,k+1)]

In [ ]:
import sympy as sp
sp.init_printing()
A = np.reshape(sp.symbols('A_1:5_1:5'),[4,4])
A = np.triu(A,1)
A += A.T
np.fill_diagonal(A,0)
A
D = np.diag(sp.symbols('k_1:5'))
L = D-A
sp.simplify(np.trace(np.linalg.matrix_power(D-A,3)))

In [ ]:
L = (sp.MatrixSymbol('D',3,3)-sp.MatrixSymbol('A',3,3))
sp.expand(L@L@L)


In [ ]:
np.trace(A@A@A)

In [ ]:
D@D@A - A@D@D

In [ ]:
A

In [ ]:
(A@D).T - D@A

In [ ]:
(A@D).T - D.T@A.T

In [ ]:
E = np.zeros([5,5])
E[3,3]=1
E

In [ ]:
R=np.random.random([5,5])
#R = R+R.T
R

In [ ]:
E@R.T@E

In [ ]:
E@R - R@E